# ...

In [1]:
%matplotlib inline
%load_ext autoreload
# use %autoreload command to reload all libraries

In [2]:
from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr"

In [3]:
# Python path fixing so we can import libraries
import sys
import os
sys_paths = ['../', # 
            ]
for p in sys_paths:
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

In [4]:
from hands.data import *
from hands.multiloss import *
from hands.hands_loss import *
from hands.notebook_utils import *
from hands.utils import * 

In [5]:
print(f"Pytorch: {torch.__version__}, Torchvision: {torchvision.__version__}, Fastai: {fastai.__version__}")

Pytorch: 1.0.0.dev20181009, Torchvision: 0.2.1, Fastai: 1.0.25.dev0


In [6]:
%autoreload

In [8]:
model_dir = "models/darknet_03"

# Setup

In [9]:
#img_org_dir = '/home/ohu/koodi/kesken/hands_data/data/jpg_exp/**/*fist/*.jpg'
img_org_dir = '/home/ohu/koodi/kesken/hands_data/data/train/**/*.jpg'
img_fnames_all = glob.glob(img_org_dir, recursive=True)
split = int(.9*len(img_fnames_all))
split = [split, len(img_fnames_all)-split]
img_fnames_arr = [list(sub) for sub in torch.utils.data.random_split(img_fnames_all, split)]

img_fnames_arr.append(glob.glob('/home/ohu/koodi/kesken/hands_data/data/test/**/*.jpg', recursive=True))

In [10]:
_ = [len(a) for a in img_fnames_arr]
_
[a/sum(_) for a in _]

[13183, 1465, 4723]

[0.6805534045738475, 0.07562851685509267, 0.24381807857105983]

# Model / Learner Setup

In [11]:
from hands.models.model_darknet_customized import *

In [12]:
model = ModelDarknetCustomized(num_classes=len(HandItems.CLASS_NAMES),
                               darknet_layers=[1,2,8], darknet_output=256)

In [13]:
_ = model.cuda().train()
learn = fastai.Learner(data=get_default_databunch(img_fnames_arr, size=320),
                       model=model, loss_func=HandsLoss())
learn.metrics = [HandsAccuracy()]
learn.callback_fns.append(HandleAvgMultiLoss)
learn.path = pathlib.Path('data/')
learn.split((model.pre_0[0][0], ))
learn.model_dir = model_dir

In [14]:
if not os.path.isdir(learn.path/learn.model_dir):
    os.mkdir(learn.path/learn.model_dir)

In [30]:
_ = model_load_backbone(model, '../data/models/yolov3_coco_01.h5')

Loaded to: backbone.base.conv.weight -> backbone.darknet_2.8.conv2.bn.running_var.
Skipped from: backbone.darknet_3.0.conv.weight...


In [31]:
learn.freeze()
model_requires_grad_info(model)

backbone.base.conv.weight: False
...
pre_0.14_convbatch.0.weight: True
...
pre_0.15_conv.bias: True


In [32]:
import gc
def empty_cuda_cache():
    for _ in range(2):
        gc.collect()
        torch.cuda.empty_cache()
        time.sleep(1)

In [33]:
empty_cuda_cache()

# Learner & Fitting

### Size 320

In [34]:
sz = 10*32
sz
learn.data = get_default_databunch(img_fnames_arr, size=sz, bs=32)

320

In [35]:
learn.fit_one_cycle(3, max_lr=1e-2)

Total time: 15:06
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle 
1      32.353931   38.123287   0.210239  3.946073   23.569960  5.610484    4.996767    (05:05)
2      20.823776   21.951010   0.362457  3.429120   12.418602  4.142219    1.961067    (05:00)
3      15.152591   15.713665   0.492150  3.288147   7.666296  3.579290    1.179934     (05:01)



In [36]:
learn.save('320_1')

In [37]:
learn.validate(learn.data.test_dl)

[48.759262, 0.390006351894982]

In [38]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.3226762650857506,
 {'not_enough_preds': 2362,
  'too_many_preds': 9,
  'wrong_class': 396,
  'too_far': 0,
  'wrong_direction': 432})

In [39]:
learn.unfreeze()

In [40]:
model_requires_grad_info(model)

backbone.base.conv.weight: True
...
pre_0.15_conv.bias: True


In [41]:
learn.fit_one_cycle(30, max_lr=1e-4)

Total time: 2:33:10
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle
1      14.425879   14.848296   0.496928  3.288843   6.838097  3.539479    1.181877    (05:06)
2      13.107965   13.994322   0.501024  3.268462   6.078667  3.481814    1.165381    (05:04)
3      12.243313   13.413573   0.489420  3.270960   5.553333  3.459597    1.129683    (05:05)
4      11.875992   12.784998   0.508532  3.234760   5.007298  3.419882    1.123058    (05:04)
5      11.468328   11.836123   0.535836  3.212100   4.193471  3.359447    1.071104    (05:07)
6      10.372625   11.637770   0.536519  3.185982   4.058462  3.329210    1.064116    (05:04)
7      10.491927   10.878450   0.554266  3.156615   3.458081  3.268347    0.995409    (05:04)
8      9.644271    10.813465   0.529010  3.124824   3.552514  3.227644    0.908483    (05:05)
9      9.164272    10.614743   0.586348  3.101395   3.468885  3.147286    0.897177    (05:06)
10     9.118854    10.256940   0.569283  3.068684

In [42]:
learn.save('320_2') # this is same as hands_darknet_03_320_v01.pth

In [43]:
learn.validate(learn.data.test_dl)

[47.642555, 0.5022231632437011]

In [44]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.46157103535888205,
 {'not_enough_preds': 1792,
  'too_many_preds': 10,
  'wrong_class': 327,
  'too_far': 1,
  'wrong_direction': 413})

### Size 416

In [49]:
_ = learn.load('320_2')

In [50]:
sz = 13*32
sz
learn.data = get_default_databunch(img_fnames_arr, size=sz, bs=16)

416

In [51]:
learn.validate(learn.data.test_dl)

[20.879303, 0.49163667160702945]

In [52]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.4393394029218717,
 {'not_enough_preds': 2041,
  'too_many_preds': 9,
  'wrong_class': 252,
  'too_far': 1,
  'wrong_direction': 345})

In [53]:
learn.fit_one_cycle(30, max_lr=1e-5)

Total time: 3:06:52
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle
1      3.599421    4.135702    0.629352  1.623667   0.499205  1.741775    0.271056    (06:11)
2      4.042902    4.102026    0.621843  1.614777   0.481438  1.735670    0.270140    (06:13)
3      3.374560    4.083979    0.633447  1.611655   0.478223  1.725975    0.268127    (06:11)
4      3.611433    4.085629    0.635495  1.604728   0.487954  1.726111    0.266835    (06:13)
5      3.356045    4.083211    0.641638  1.602282   0.509253  1.708875    0.262801    (06:12)
6      3.722077    4.065437    0.632765  1.594273   0.509201  1.699214    0.262748    (06:13)
7      3.486089    4.068674    0.647099  1.596943   0.520293  1.695648    0.255791    (06:12)
8      3.251864    4.018857    0.640273  1.596034   0.477926  1.690276    0.254621    (06:12)
9      3.365352    4.034848    0.634812  1.577145   0.510395  1.690490    0.256818    (06:13)
10     3.493430    3.915245    0.634130  1.568752

In [54]:
learn.save('416_1') # this is copied to hands_darknet_full_608_v01.pth

In [55]:
learn.validate(learn.data.test_dl)

[21.744856, 0.49100148210882916]

In [56]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size, max_samples=0)
corr, wrongs

(0.439974592420072,
 {'not_enough_preds': 2055,
  'too_many_preds': 0,
  'wrong_class': 256,
  'too_far': 0,
  'wrong_direction': 334})

In [57]:
learn.fit_one_cycle(30, max_lr=1e-6)

Total time: 3:07:06
epoch  train_loss  valid_loss  accuracy  loss_conf  loss_cls  loss_coord  loss_angle
1      3.142807    3.732712    0.664164  1.534974   0.380013  1.596005    0.221720    (06:12)
2      3.073757    3.733470    0.662799  1.535124   0.380288  1.595935    0.222123    (06:12)
3      3.065186    3.730651    0.662799  1.534264   0.379054  1.596316    0.221017    (06:13)
4      3.004814    3.724472    0.662116  1.532081   0.376378  1.595911    0.220103    (06:13)
5      2.907373    3.728948    0.665529  1.535206   0.376660  1.595457    0.221625    (06:11)
6      2.869380    3.726637    0.660068  1.533224   0.377619  1.594966    0.220828    (06:13)
7      3.133643    3.728704    0.663481  1.534346   0.378618  1.594109    0.221630    (06:12)
8      3.114626    3.738975    0.665529  1.536062   0.390040  1.591478    0.221394    (06:13)
9      2.907002    3.726834    0.661433  1.533480   0.381474  1.590480    0.221400    (06:13)
10     3.191125    3.728314    0.658703  1.532773

In [58]:
learn.save('416_2')  # this is same as hands_darknet_03_416_v01.pth

In [59]:
learn.validate(learn.data.test_dl)

[21.658089, 0.49227186110522975]

In [60]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size)
corr, wrongs

(0.439974592420072,
 {'not_enough_preds': 2049,
  'too_many_preds': 0,
  'wrong_class': 256,
  'too_far': 0,
  'wrong_direction': 340})

**Note we can get much more accuracy if we lower the confidence threshold. But then we'll also get a lot of more of wrong predictions**

In [62]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size,
                                                           get_results_args={'conf_thresh': .20})
corr, wrongs

(0.5604488672453949,
 {'not_enough_preds': 1069,
  'too_many_preds': 38,
  'wrong_class': 471,
  'too_far': 4,
  'wrong_direction': 494})

In [63]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size,
                                                           get_results_args={'conf_thresh': .10})
corr, wrongs

(0.5941139106500106,
 {'not_enough_preds': 484,
  'too_many_preds': 247,
  'wrong_class': 624,
  'too_far': 3,
  'wrong_direction': 559})

In [64]:
corr, wrongs, w_samples, w_idxs = calculate_accuracy_files(model, learn.data.test_ds.fnames, learn.data.test_dl.size,
                                                           get_results_args={'conf_thresh': .05})
corr, wrongs

(0.46919330933728565,
 {'not_enough_preds': 156,
  'too_many_preds': 1312,
  'wrong_class': 569,
  'too_far': 5,
  'wrong_direction': 465})